## Описание робота RHANTOM 1.5

In [1]:
import numpy as np

In [4]:
class RobotIface:
    '''
        Robot's dynamics:
        Inertia(state)*state'' + Coriolis(state)*state' + Gravity(state) = tau
    '''
    def inertia(self, state:np.array) -> np.matrix:
        pass
    def coriolis(self, state:np.array) -> np.matrix:
        pass
    def gravity(self, state:np.array) -> np.matrix:
        pass
    '''
        Robot's regression dinamics:
        Inertia(state, param)*state'' + Coriolis(state, param)*state' + Gravity(state, param) = regressor(state, state', state'') * param
    '''
    def initial_params(self) -> np.array:
        pass
    def inertia_p(self, state:np.array, param:np.array) -> np.matrix:
        pass
    def coriolis_p(self, state:np.array, param:np.array) -> np.matrix:
        pass
    def gravity_p(self, state:np.array, param:np.array) -> np.matrix:
        pass
    def regressor(self, state:np.array, d_state:np.array, dd_state: np.array) -> np.matrix:
        pass
    '''
        Robot's kinematics
    '''
    def forward_kinematics(self, state:np.array) -> np.array:
        pass
    def jacobian(self, state:np.array) -> np.matrix:
        pass
    def d_jacobian(self, state:np.array) -> np.matrix:
        pass

In [6]:
class Phantom(RobotIface):
    l = [0.215, 0.170, 0.0325, 0, -0.0368, 0.0527]
    m_a  = 0.0202
    m_c  = 0.0249
    m_be = 0.2359
    m_df = 0.1906
    i_axx = 0.4864/10000
    i_ayy = 0.0018/10000
    i_azz = 0.4864/10000
    i_cxx = 0.959/10000
    i_cyy = 0.959/10000
    i_czz = 0.0051/10000
    i_bexx = 11.09/10000
    i_beyy = 10.06/10000
    i_bezz = 0.591/10000
    i_dfxx = 7.11/10000
    i_dfyy = 0.629/10000
    i_dfzz = 6.246/10000
    i_baseyy = 11.87/10000
    g = 9.81

    def inertia(self, state:np.array) -> np.matrix:
        result = np.zeros((3, 3))
        result[0, 0] = 1/8*(4*self.i_ayy+4*self.i_azz+8*self.i_baseyy+4*self.i_beyy+4*self.i_bezz+4*self.i_cyy+4*self.i_czz +4*self.i_dfyy+4*self.i_dfzz+4*self.l[0]**2*self.m_a+self.l[1]**2*self.m_a+self.l[0]**2*self.m_c+4*self.l[2]**2*self.m_c)+\
                1/8*(4*self.i_beyy-4*self.i_bezz+4*self.i_cyy-4*self.i_czz+self.l[0]**2*(4*self.m_a+self.m_c))*np.cos(2*state[1])+\
                1/8*(4*self.i_ayy-4*self.i_azz+4*self.i_dfyy-4*self.i_dfzz-self.l[1]**2*self.m_a-4*self.l[2]**2*self.m_c)*np.cos(2*state[2])+\
                self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*np.cos(state[1])*np.sin(state[2])
        result[1, 1] = 1/4*(4*(self.i_bexx+self.i_cxx+self.l[0]**2*self.m_a)+self.l[0]**2*self.m_c)
        result[1, 2] = -1/2*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*np.sin(state[1]-state[2])
        result[2, 1] = result[1, 2]
        result[2, 2] = 1/4*(4*self.i_axx+4*self.i_dfxx+self.l[1]**2*self.m_a+4*self.l[2]**2*self.m_c)
        return result

    def coriolis(self, state:np.array) -> np.matrix:
        s2 = np.sin(state[1]); c2 = np.cos(state[1])
        s3 = np.sin(state[2]); c3 = np.cos(state[2])

        result = np.zeros((3, 3))
        result[0, 0] = 1/8*(-2*s2*((4*self.i_beyy-4*self.i_bezz+4*self.i_cyy-4*self.i_czz+4*self.l[0]**2*self.m_a+self.l[0]**2*self.m_c)*c2+2*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*s3)*state[4]+2*c3*(2*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*c2+(-4*self.i_ayy+4*self.i_azz-4*self.i_dfyy+4*self.i_dfzz+self.l[1]**2*self.m_a+4*self.l[2]**2*self.m_c)*s3)*state[5])
        result[0, 1] = -1/8*((4*self.i_beyy-4*self.i_bezz+4*self.i_cyy-4*self.i_czz+self.l[0]**2*(4*self.m_a+self.m_c))*s2+4*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*s2*s3)*state[3]
        result[1, 0] = -result[0, 1]
        result[0, 2] = 1/8*(4*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*c2*c3+(-4*self.i_ayy+4*self.i_azz-4*self.i_dfyy+4*self.i_dfzz+self.l[1]**2*self.m_a+4*self.l[2]**2*self.m_c)*np.sin(2*state[2]))*state[3]
        result[2, 0] = -result[0, 2]
        result[1, 2] = 1/2*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*np.cos(state[1]-state[2])*state[5]
        result[2, 1] = 1/2*self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)*np.cos(state[1]-state[2])*state[4]
        return result

    def gravity(self, state:np.array) -> np.matrix:
        result = np.zeros((3, 1))
        result[1, 0] = 1/2*self.g*(2*self.l[0]*self.m_a+2*self.l[4]*self.m_be+self.l[0]*self.m_c)*np.cos(state[1])
        result[2, 0] = 1/2*self.g*(self.l[1]*self.m_a+2*self.l[2]*self.m_c-2*self.l[5]*self.m_df)*np.sin(state[2])
        return result

    def initial_params(self) -> np.array:
        result = np.zeros(8)
        result[0] = 1/8*(4*self.i_ayy+4*self.i_azz+8*self.i_baseyy+4*self.i_beyy+4*self.i_bezz+4*self.i_cyy+4*self.i_czz +4*self.i_dfyy+4*self.i_dfzz+4*self.l[0]**2*self.m_a+self.l[1]**2*self.m_a+self.l[0]**2*self.m_c+4*self.l[2]**2*self.m_c)
        result[1] = 1/8*(4*self.i_beyy-4*self.i_bezz+4*self.i_cyy-4*self.i_czz+self.l[0]**2*(4*self.m_a+self.m_c))
        result[2] = 1/8*(4*self.i_ayy-4*self.i_azz+4*self.i_dfyy-4*self.i_dfzz-self.l[1]**2*self.m_a-4*self.l[2]**2*self.m_c)
        result[3] = self.l[0]*(self.l[1]*self.m_a+self.l[2]*self.m_c)
        result[4] = 1/4*(4*(self.i_bexx+self.i_cxx+self.l[0]**2*self.m_a)+self.l[0]**2*self.m_c)
        result[5] = 1/4*(4*self.i_axx+4*self.i_dfxx+self.l[1]**2*self.m_a+4*self.l[2]**2*self.m_c)
        result[6] = 1/2*self.g*(2*self.l[0]*self.m_a+2*self.l[4]*self.m_be+self.l[0]*self.m_c)
        result[7] = 1/2*self.g*(self.l[1]*self.m_a+2*self.l[2]*self.m_c-2*self.l[5]*self.m_df)
        return result

    def inertia_p(self, state:np.array, p:np.array):
        result = np.zeros((3, 3))
        result[0, 0] = p[0]+p[1]*np.cos(2*state[1])+p[2]*np.cos(2*state[2])+p[3]*np.cos(state[1])*np.sin(state[2])
        result[1, 1] = p[4]
        result[1, 2] = -1/2*p[3]*np.sin(state[1]-state[2])
        result[2, 1] = result[1, 2]
        result[2, 2] = p[5]
        return result

    def coriolis_p(self, state:np.array, p:np.array):
        c2 = np.cos(state[1]); s2 = np.sin(state[1])
        c3 = np.cos(state[2]); s3 = np.sin(state[2])

        result = np.zeros((3, 3))
        result[0, 0] = 1/8*(-2*s2*(8*p[1]*c2+2*p[3]*np.sin(state[2]))*state[4]+2*c3*(2*p[3]*c2-8*p[2]*s3)*state[5])
        result[0, 1] = -1/8*(8*p[1]*s2+4*p[3]*s2*s3)*state[3]
        result[1, 0] = -result[0, 1]
        result[0, 2] = 1/8*(4*p[3]*c2*c3-8*p[2]*np.sin(2*state[2]))*state[3]
        result[2, 0] = -result[0, 2]
        result[1, 2] = 1/2*p[3]*np.cos(state[1]-state[2])*state[5]
        result[2, 1] = 1/2*p[3]*np.cos(state[1]-state[2])*state[4]
        return result
    
    def gravity_p(self, state:np.array, p:np.array):
        result = np.zeros((3, 1))
        result[1, 0] = p[6]*np.cos(state[1])
        result[2, 0] = p[7]*np.sin(state[2])
        return result

    def regressor(self, q, dq, d2q):
        Y = np.zeros((8, 3))
        Y[0, 0] = d2q[0]
        Y[1, 0] = d2q[0]*np.cos(2*q[1]) - 2*dq[0]*dq[1]*np.sin(q[1])*np.cos(q[1]) - dq[0]*dq[1]*np.sin(2*q[1])
        Y[1, 1] = dq[0]**2*np.sin(2*q[1])
        Y[2, 0] = d2q[0]*np.cos(2*q[2]) - 2*dq[0]*dq[2]*np.sin(q[2])*np.cos(q[2]) - dq[0]*dq[2]*np.sin(2*q[2])
        Y[2, 2] = dq[0]**2*np.sin(2*q[2])
        Y[3, 0] = d2q[0]*np.cos(q[1])*np.sin(q[2]) - dq[0]*dq[1]*np.sin(q[1])*np.sin(q[2]) + dq[0]*dq[2]*np.cos(q[1])*np.cos(q[2])
        Y[3, 1] = -(1/2)*d2q[2]*np.sin(q[1]-q[2]) + (1/2)*dq[0]**2*np.sin(q[1])*np.sin(q[2]) + (1/2)*dq[2]**2*np.cos(q[1]-q[2])
        Y[3, 2] = -(1/2)*d2q[1]*np.sin(q[1]-q[2]) - (1/2)*dq[0]**2*np.cos(q[1])*np.cos(q[2]) + (1/2)*dq[1]**2*np.cos(q[1]-q[2])
        Y[4, 1] = d2q[1]
        Y[5, 2] = d2q[2]
        Y[6, 1] = np.cos(q[1])
        Y[7, 2] = np.sin(q[2])
        
        inv_jacobian = np.linalg.inv(self.jacobian(q))
        return inv_jacobian.T @ Y.T

    def forward_kinematics(self, state:np.array) -> np.array:
        c1 = np.cos(state[0]); c2 = np.cos(state[1]); c3 = np.cos(state[2])
        s1 = np.sin(state[0]); s2 = np.sin(state[1]); s3 = np.sin(state[2])
        
        result = np.zeros(3)
        result[0] = s1*(self.l[1]*s3 + self.l[0]*c2)
        result[1] = self.l[1]*(1-c3)+self.l[0]*s2
        result[2] = -self.l[0]+c1*(self.l[1]*s3 + self.l[0]*c2)
        return result
    
    def jacobian(self, state:np.array)->np.matrix:
        c1 = np.cos(state[0]); c2 = np.cos(state[1]); c3 = np.cos(state[2])
        s1 = np.sin(state[0]); s2 = np.sin(state[1]); s3 = np.sin(state[2])

        result = np.zeros((3, 3))
        result[0, 0] = c1*(self.l[1]*s3+self.l[0]*c2)
        result[0, 1] = -self.l[0]*s2*s1
        result[0, 2] = self.l[1]*c3*s1
        result[1, 1] = self.l[0]*c2
        result[1, 2] = self.l[1]*s3
        result[2, 0] = -s1*(self.l[1]*s3+self.l[0]*c2)
        result[2, 1] = -self.l[0]*s2*c1
        result[2, 2] = self.l[1]*c3*c1
        return result
    
    def d_jacobian(self, state:np.array)->np.matrix:
        c1 = np.cos(state[0]); s1 = np.sin(state[0])
        c2 = np.cos(state[1]); s2 = np.sin(state[1])
        c3 = np.cos(state[2]); s3 = np.sin(state[2])

        dj1 = np.array([[-(self.l[1]*s3 + self.l[0]*c2)*s1, -self.l[0]*s2*c1, self.l[1]*c1*c3],
                   [0, 0, 0],
                   [-(self.l[1]*s3 + self.l[0]*c2)*c1, self.l[0]*s1*s2,  -self.l[1]*s1*c3]])
        dj2 = np.array([[-self.l[0]*s2*c1, -self.l[0]*s1*c2, 0],
                   [0, -self.l[0]*s2, 0],
                   [self.l[0]*s1*s2, -self.l[0]*c1*c2, 0]])
        dj3 = np.array([[self.l[1]*c1*c3, 0, -self.l[1]*s1*s3],
                   [0, 0, self.l[1]*c3], 
                   [-self.l[1]*s1*c3, 0, -self.l[1]*s3*c1]])
        return dj1*state[3] + dj2*state[4] + dj3*state[5]